In [1]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple

# Лабораторная работа №2. Методы поиска

Вариант: 15

## Алгоритм Ахо — Корасик(Aho – Corasick algorithm)

### Классификация алгоритма:

- По типу алгоритма: алгоритм поиска подстроки

- По устойчивости: устойчивый

- По месту хранения данных: выделяет память для создания бора и инициализации узлов

- По выделению дополнительного места: использует

- По дополнительным затратам памяти: 
  - Если таблицу переходов автомата хранить как индексный массив: $O(nq)$
  
  - Если таблицу переходов автомата хранить как красно-черное дерево: $O(n)$
   
- Время выполнения: 
  - Индексный массив: $O(nq + H + k)$, где H - длина текста, n - общая длина слов в словаре, q - размер алфавита и k - общая длина всех совпадений
    
  -  Красно-черное дерево: $O((H + n)logq + k)$

### Описание алгоритма:

Алгоритм строит конечный автомат, которому затем передаёт строку поиска. Автомат получает по очереди все символы строки и переходит по соответствующим рёбрам. Если автомат пришёл в конечное состояние, соответствующая строка словаря присутствует в строке поиска.

Несколько строк поиска можно объединить в дерево поиска, так называемый бор (префиксное дерево). Бор является конечным автоматом, распознающим одну строку из m — но при условии, что начало строки известно.

Первая задача в алгоритме — научить автомат «самовосстанавливаться», если подстрока не совпала. При этом перевод автомата в начальное состояние при любой неподходящей букве не подходит, так как это может привести к пропуску подстроки (например, при поиске строки aabab, попадается aabaabab, после считывания пятого символа перевод автомата в исходное состояние приведёт к пропуску подстроки — верно было бы перейти в состояние a, а потом снова обработать пятый символ). Чтобы автомат самовосстанавливался, к нему добавляются суффиксные ссылки, нагруженные пустым символом ⌀ (так что детерминированный автомат превращается в недетерминированный). Например, если разобрана строка aaba, то бору предлагаются суффиксы aba, ba, a. Суффиксная ссылка — это ссылка на узел, соответствующий самому длинному суффиксу, который не заводит бор в тупик (в данном случае a).

Для корневого узла суффиксная ссылка — петля. Для остальных правило таково: если последний распознанный символ — c, то осуществляется обход по суффиксной ссылке родителя, если оттуда есть дуга, нагруженная символом c, суффиксная ссылка направляется в тот узел, куда эта дуга ведёт. Иначе — алгоритм проходит по суффиксной ссылке ещё и ещё раз, пока либо не пройдёт по корневой ссылке-петле, либо не найдёт дугу, нагруженную символом c.

![img](./src/diagram.svg.png)

### Блок-схема алгоритма

![img](./src/Aho-Korasik.png)

### Псевдокод

In [ ]:
function build_trie(patterns):
    trie = create_empty_trie()
    for pattern in patterns:
        add_pattern_to_trie(trie, pattern)
    return trie

function build_aho_corasick(trie):
    queue = initialize_queue(trie)
    while queue is not empty:
        node = queue.pop()
        for edge in node.edges:
            child = edge.target
            queue.push(child)
            calculate_suffix_link(trie, child, edge.symbol)
            calculate_output(trie, child)

function search_aho_corasick(text, trie):
    state = trie.root
    for i in range(len(text)):
        state = get_next_state(trie, state, text[i])
        if state.output:
            for pattern in state.output:
                print("found pattern", pattern, "at index", i - len(pattern) + 1)

### Достоинства и недостатки

Достоинства:

- Эффективный поиск множества подстрок в строке (линейное время)
- Не требует дополнительной памяти для хранения промежуточных результатов
- Однократное сканирование текста

Недостатки:

- Построение бора и автомата может занять много времени и памяти
- Не эффективен для поиска одной подстроки

### Реализация алгоритма

In [2]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.output = []
        self.fail = None

class AhoCorasick:
    def __init__(self, patterns):
        self.root = TrieNode()
        self.build_trie(patterns)
        self.build_automaton()

    def build_trie(self, patterns):
        for pattern in patterns:
            node = self.root
            for char in pattern:
                if char not in node.children:
                    node.children[char] = TrieNode()
                node = node.children[char]
            node.output.append(pattern)

    def build_automaton(self):
        queue = [self.root]
        while queue:
            node = queue.pop(0)
            for char, child in node.children.items():
                queue.append(child)
                fail = node.fail
                while fail and char not in fail.children:
                    fail = fail.fail
                child.fail = fail.children[char] if fail else self.root
                child.output.extend(child.fail.output)

    def search(self, text):
        results = []
        node = self.root
        for i, char in enumerate(text):
            while node and char not in node.children:
                node = node.fail
            if not node:
                node = self.root
            else:
                node = node.children[char]
                for pattern in node.output:
                    results.append((i - len(pattern) + 1, pattern))
        return results

### Демонстрация работы алгоритма

In [3]:
patterns = ["a", "ab", "abc", "bc", "c", "cba"]
aho_corasick = AhoCorasick(patterns)
text = "abcdaabaacbd"
results = aho_corasick.search(text)

for index, pattern in results:
    print(f"Found pattern '{pattern}' at index {index}")

Found pattern 'a' at index 0
Found pattern 'ab' at index 0
Found pattern 'abc' at index 0
Found pattern 'bc' at index 1
Found pattern 'c' at index 2
Found pattern 'a' at index 4
Found pattern 'a' at index 5
Found pattern 'ab' at index 5
Found pattern 'a' at index 7
Found pattern 'a' at index 8
Found pattern 'c' at index 9


### Тестирование

In [4]:
patterns = ["a", "ab", "bc", "aab", "aac", "bd"]
aho_corasick = AhoCorasick(patterns)

# Тест 1: все образцы найдены
text = "abcdaabaacbd"
results = aho_corasick.search(text)
print(f"Тест 1: {results}")
assert len(results) == 11

# Тест 2: нет совпадений
text = "defg"
results = aho_corasick.search(text)
print(f"Тест 2: {results}")
assert len(results) == 0

# Тест 3: найдены только некоторые образцы
text = "abcdeaac"
results = aho_corasick.search(text)
print(f"Тест 3: {results}")
assert len(results) == 6

# Тест 4: строка содержит повторяющиеся образцы
text = "ababa"
results = aho_corasick.search(text)
print(f"Тест 4: {results}")
assert len(results) == 5

# Тест 5: строка состоит только из одного образца
text = "a"
results = aho_corasick.search(text)
print(f"Тест 5: {results}")
assert len(results) == 1

print("All tests passed!")

Тест 1: [(0, 'a'), (0, 'ab'), (1, 'bc'), (4, 'a'), (5, 'a'), (4, 'aab'), (5, 'ab'), (7, 'a'), (8, 'a'), (7, 'aac'), (10, 'bd')]
Тест 2: []
Тест 3: [(0, 'a'), (0, 'ab'), (1, 'bc'), (5, 'a'), (6, 'a'), (5, 'aac')]
Тест 4: [(0, 'a'), (0, 'ab'), (2, 'a'), (2, 'ab'), (4, 'a')]
Тест 5: [(0, 'a')]
All tests passed!


### Все вхождения имени главного героя в любимом литературном произведении

In [5]:
# Загрузить текст книги
with open("./src/Gambrinus.txt", encoding="utf-8") as file:
    book_text = file.read()

# Создать экземпляр алгоритма с именем "Сашка" в качестве образца
patterns = ["Сашка"]
aho_corasick = AhoCorasick(patterns)

# Применить алгоритм к тексту книги
results = aho_corasick.search(book_text)

# Вывести результаты
print(f"Имя 'Сашка' найдено {len(results)} раз.")
for result in results:
    print(f"Позиция: {result[0]}, Совпадение: {result[1]}")

Имя 'Сашка' найдено 56 раз.
Позиция: 3079, Совпадение: Сашка
Позиция: 3322, Совпадение: Сашка
Позиция: 9579, Совпадение: Сашка
Позиция: 10016, Совпадение: Сашка
Позиция: 10377, Совпадение: Сашка
Позиция: 10595, Совпадение: Сашка
Позиция: 11911, Совпадение: Сашка
Позиция: 12222, Совпадение: Сашка
Позиция: 13813, Совпадение: Сашка
Позиция: 13941, Совпадение: Сашка
Позиция: 14056, Совпадение: Сашка
Позиция: 14079, Совпадение: Сашка
Позиция: 14112, Совпадение: Сашка
Позиция: 14246, Совпадение: Сашка
Позиция: 14739, Совпадение: Сашка
Позиция: 15080, Совпадение: Сашка
Позиция: 16153, Совпадение: Сашка
Позиция: 17091, Совпадение: Сашка
Позиция: 17299, Совпадение: Сашка
Позиция: 17708, Совпадение: Сашка
Позиция: 18362, Совпадение: Сашка
Позиция: 20994, Совпадение: Сашка
Позиция: 21175, Совпадение: Сашка
Позиция: 22393, Совпадение: Сашка
Позиция: 23461, Совпадение: Сашка
Позиция: 25073, Совпадение: Сашка
Позиция: 25990, Совпадение: Сашка
Позиция: 26009, Совпадение: Сашка
Позиция: 28733, Совпаде

## Алгоритм имитации отжига(Simulated annealing)

### Классификация алгоритма:

- По типу алгоритма: гибридный

- По устойчивости: неустойчивый

- По месту хранения данных: может хранить данные на месте или выделять дополнительное пространство

- По выделению дополнительного места: использует

- По адаптивности: может быть адаптивным, если использует изменяемые параметры. в остальных случаях - неадаптивный
   
- Время выполнения: 
  - В худшем случае: $O(2^n)$
    
  - В среднем: $O(m * n)$
  
  - В лучшем случае: $O(logn)$

### Описание алгоритма:
Алгоритм основывается на имитации физического процесса, который происходит при кристаллизации вещества, в том числе при отжиге металлов. 
Предполагается, что атомы вещества уже почти выстроены в кристаллическую решётку, но ещё допустимы переходы отдельных атомов из одной ячейки в другую. 
Активность атомов тем больше, чем выше температура, которую постепенно понижают, что приводит к тому, что вероятность переходов в состояния с большей энергией уменьшается. Устойчивая кристаллическая решётка соответствует минимуму энергии атомов, поэтому атом либо переходит в состояние с меньшим уровнем энергии, либо остаётся на месте. (Этот алгоритм также называется алгоритмом Н. Метрополиса, по имени его автора).

![SegmentLocal](src/Annealing.gif "segment")

## Блок-схема алгоритма

![img](./src/Annealing.png)

## Псевдокод

In [ ]:
current_state := initial_state()
current_energy := energy(current_state)
T := initial_temperature()
T_min := final_temperature()
L := iterations_per_temperature()
alpha := cooling_factor()
iteration_counter := 0
best_state := current_state
best_energy := current_energy

while T > T_min do
    for i in range(L) do
        new_state := random_move(current_state)
        new_energy := energy(new_state)
        if new_energy < current_energy do
            current_state := new_state
            current_energy := new_energy
        end if
        else do
            probability := exp((current_energy - new_energy) / T)
            if random() < probability do
                current_state := new_state
                current_energy := new_energy
            end if
    end for
    if current_energy < best_energy do
        best_state := current_state
        best_energy := current_energy
    end if
    T := T * alpha
    iteration_counter += L
end while

return best_state, best_energy

## Достоинства и недостатки

Достоинства:

- Может работать на любых типах задач и не требует знания градиентов
- Является глобальной оптимизационной стратегией, что позволяет находить глобальные экстремумы в сложных функциях
- Может работать в реальном времени и адаптироваться к изменениям в функции стоимости

Недостатки:

- Требуется настройка большого количества параметров, таких как начальная температура, скорость охлаждения и длина каждой итерации
- Не гарантирует нахождение оптимального решения, только приближение к нему
- Время работы может быть довольно длительным для сложных задач, особенно при больших размерностях

## Реализация алгоритма

In [6]:
def simulated_annealing(initial_state, cost_func, neighbors_func, temperature, cooling_rate):
    current_state = initial_state
    current_cost = cost_func(current_state)
    best_state = current_state
    best_cost = current_cost
    while temperature > 0.1:
        neighbor_state = random.choice(neighbors_func(current_state))
        neighbor_cost = cost_func(neighbor_state)
        delta_cost = neighbor_cost - current_cost
        if delta_cost < 0 or math.exp(-delta_cost / temperature) > random.random():
            current_state = neighbor_state
            current_cost = neighbor_cost
        if current_cost < best_cost:
            best_state = current_state
            best_cost = current_cost
        temperature *= 1 - cooling_rate
    return best_state, best_cost

initial_state = [0, 1, 2, 3, 4]

# определение функции стоимости
def cost_func(state):
    return sum(state)

# определение функции соседних состояний
def neighbors_func(state):
    neighbors = []
    for i in range(len(state)):
        for j in range(i+1, len(state)):
            neighbor = state.copy()
            neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
            neighbors.append(neighbor)
    return neighbors

# запуск алгоритма имитации отжига
best_state, best_cost = simulated_annealing(initial_state, cost_func, neighbors_func, 1000, 0.03)

print("Лучшее состояние:", best_state)
print("Лучшая стоимость:", best_cost)

Лучшее состояние: [0, 1, 2, 3, 4]
Лучшая стоимость: 10


## Тестирование

In [7]:
def test_simulated_annealing():
    def cost_func(x): # Тест на квадратичной функции
        return x**2
    
    def neighbors_func(x): # Тест на ф-ции соседних состояний
        return [x-1, x+1]

    initial_state = 5

    best_state, best_cost = simulated_annealing(initial_state, cost_func, neighbors_func, 100, 0.03)

    assert abs(best_state) < 0.1 #Ошибка при тестировании на квадратичной функции: best_state слишком далеко от минимума

    def cost_func(x): # Тест на ф-ции с множеством локальных минимумов
        return math.sin(5 * x) + math.sin(x)

    def neighbors_func(x): # Ф-ция соседних состояний
        return [x + random.uniform(-0.1, 0.1)]

    initial_state = random.uniform(-10, 10)

    best_state, best_cost = simulated_annealing(initial_state, cost_func, neighbors_func, 1000, 0.03)

    assert abs(best_state - (-1.39)) < 0.1 #Ошибка при тестировании на функции с множеством локальных минимумов: best_state слишком далеко от глобального минимума
    
    # Тест на задаче коммивояжера
    distances = [
        [0, 2, 5, 7],
        [2, 0, 6, 3],
        [5, 6, 0, 1],
        [7, 3, 1, 0]
    ]

    def cost_func(route): # Ф-ция стоимости
        cost = 0
        for i in range(len(route)):
            cost += distances[route[i-1]][route[i]]
        return cost

    def neighbors_func(route): # Ф-ция соседних состояний
        neighbors = []
        for i in range(len(route)):
            for j in range(i+1, len(route)):
                neighbor = route.copy()
                neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
                neighbors.append(neighbor)
        return neighbors

    initial_state

## Поиск экстремума функции

In [8]:
def simulated_annealing(initial_state, cost_func, neighbors_func, temperature, cooling_rate):
    current_state = initial_state
    current_cost = cost_func(current_state)
    best_state = current_state
    best_cost = current_cost
    while temperature > 0.1:
        neighbor_state = random.choice(neighbors_func(current_state))
        neighbor_cost = cost_func(neighbor_state)
        delta_cost = neighbor_cost - current_cost
        if delta_cost < 0 or math.exp(-delta_cost / temperature) > random.random():
            current_state = neighbor_state
            current_cost = neighbor_cost
        if current_cost < best_cost:
            best_state = current_state
            best_cost = current_cost
        temperature *= 1 - cooling_rate
    return best_state, best_cost

# Определение ф-ции стоимости
def cost_func(x):
    return 12*x**3+12*x**2

# Определение ф-ции соседних состояний
def neighbors_func(x):
    delta = 0.1
    return [x + delta, x - delta]

# Запуск алгоритма имитации отжига
initial_state = 0
best_state, best_cost = simulated_annealing(initial_state, cost_func, neighbors_func, 1000, 0.03)

print("Лучшее состояние: ", best_state)
print("Лучшая стоимость: ", best_cost)

Лучшее состояние:  -4.999999999999998
Лучшая стоимость:  -1199.9999999999986


## Литература

Дональд Э. Кнут. Искусство программирования, том 2. Получисленные алгоритмы = The Art of Computer Programming, vol.2. Seminumerical Algorithms, 3-ed. — Вильямс, 2007. — С. 832. — ISBN 978-5-8459-0081-4.

Роберт Седжвик. Фундаментальные алгоритмы на C. Анализ/Структуры данных/Сортировка/Поиск = Algorithms in C. Fundamentals/Data Structures/Sorting/Searching. — СПб.: ДиаСофтЮП, 2003. — С. 672. — ISBN 5-93772-081-4.